<a href="https://colab.research.google.com/github/gkadusumilli/Voxelnet/blob/master/VoxelNet_data_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Clone the voxelnet repo

In [ ]:
!git clone https://github.com/gkadusumilli/Voxelnet.git

Cloning into 'Voxelnet'...
remote: Enumerating objects: 80, done.
remote: Total 80 (delta 0), reused 0 (delta 0), pack-reused 80
Unpacking objects: 100% (80/80), done.


In [ ]:
%cd /content/Voxelnet

/content/Voxelnet


##Build the dependencies

In [ ]:
!python setup.py build_ext --inplace

Compiling ./utils/box_overlaps.pyx because it changed.
[1/1] Cythonizing ./utils/box_overlaps.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/Voxelnet/utils/box_overlaps.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'utils.box_overlaps' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/utils
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -I/usr/include/python3.6m -c ./utils/box_overlaps.c -o build/temp.linux-x86_64-3.6/./utils/box_overlaps.o
In file included from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1832:0,
                 from /usr/loc

In [ ]:
%cd /content/Voxelnet/crop_data

/content/Voxelnet/crop_data


##Velodyne data

In [ ]:
#register to the KITTI dataset, link will be mailed
!wget  (your link)

In [ ]:
!unzip /content/Voxelnet/crop_data/data_object_velodyne.zip 'training/*' -d /content/Voxelnet/crop_data

##Image

In [ ]:
!wget (your link)

In [ ]:
!unzip /content/Voxelnet/crop_data/data_object_image_2.zip 'training/*' -d /content/Voxelnet/crop_data

##label-2

In [ ]:
!wget  (your link)

In [ ]:
!unzip /content/Voxelnet/crop_data/data_object_label_2.zip

##Calib file

In [ ]:
!wget  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_calib.zip

In [ ]:
!unzip /content/Voxelnet/crop_data/data_object_calib.zip 'training/*' -d /content/Voxelnet/crop_data

cropping point cloud data for training and validation. Point clouds outside the image coordinates are removed. cropped point cloud will overwrite the existing raw point cloud

### To edit the path directories in the crop.py (optional)

In [ ]:
#command to view the .py file 
%pycat /content/Voxelnet/crop_data/crop.py

In [ ]:
#Copy the code edit the file (optional)
%%writefile /content/Voxelnet/crop_data/crop.py

import numpy as np
from scipy.misc import imread

CAM = 2

def load_velodyne_points(filename):
    points = np.fromfile(filename, dtype=np.float32).reshape(-1, 4)
    #points = points[:, :3]  # exclude luminance
    return points

def load_calib(calib_dir):
    # P2 * R0_rect * Tr_velo_to_cam * y
    lines = open(calib_dir).readlines()
    lines = [ line.split()[1:] for line in lines ][:-1]
    #
    P = np.array(lines[CAM]).reshape(3,4)
    #
    Tr_velo_to_cam = np.array(lines[5]).reshape(3,4)
    Tr_velo_to_cam = np.concatenate(  [ Tr_velo_to_cam, np.array([0,0,0,1]).reshape(1,4)  ]  , 0     )
    #
    R_cam_to_rect = np.eye(4)
    R_cam_to_rect[:3,:3] = np.array(lines[4][:9]).reshape(3,3)
    #
    P = P.astype('float32')
    Tr_velo_to_cam = Tr_velo_to_cam.astype('float32')
    R_cam_to_rect = R_cam_to_rect.astype('float32')
    return P, Tr_velo_to_cam, R_cam_to_rect

def prepare_velo_points(pts3d_raw):
    '''Replaces the reflectance value by 1, and tranposes the array, so
        points can be directly multiplied by the camera projection matrix'''
    pts3d = pts3d_raw
    # Reflectance > 0
    indices = pts3d[:, 3] > 0
    pts3d = pts3d[indices ,:]
    pts3d[:,3] = 1
    return pts3d.transpose(), indices

def project_velo_points_in_img(pts3d, T_cam_velo, Rrect, Prect):
    '''Project 3D points into 2D image. Expects pts3d as a 4xN
        numpy array. Returns the 2D projection of the points that
        are in front of the camera only an the corresponding 3D points.'''
    # 3D points in camera reference frame.
    pts3d_cam = Rrect.dot(T_cam_velo.dot(pts3d))
    # Before projecting, keep only points with z>0
    # (points that are in fronto of the camera).
    idx = (pts3d_cam[2,:]>=0)
    pts2d_cam = Prect.dot(pts3d_cam[:,idx])
    return pts3d[:, idx], pts2d_cam/pts2d_cam[2,:], idx


def align_img_and_pc(img_dir, pc_dir, calib_dir):
    
    img = imread(img_dir)
    pts = load_velodyne_points( pc_dir )
    P, Tr_velo_to_cam, R_cam_to_rect = load_calib(calib_dir)

    pts3d, indices = prepare_velo_points(pts)
    pts3d_ori = pts3d.copy()
    reflectances = pts[indices, 3]
    pts3d, pts2d_normed, idx = project_velo_points_in_img( pts3d, Tr_velo_to_cam, R_cam_to_rect, P  )
    #print reflectances.shape, idx.shape
    reflectances = reflectances[idx]
    #print reflectances.shape, pts3d.shape, pts2d_normed.shape
    assert reflectances.shape[0] == pts3d.shape[1] == pts2d_normed.shape[1]

    rows, cols = img.shape[:2]

    points = []
    for i in range(pts2d_normed.shape[1]):
        c = int(np.round(pts2d_normed[0,i]))
        r = int(np.round(pts2d_normed[1,i]))
        if c < cols and r < rows and r > 0 and c > 0:
            color = img[r, c, :]
            point = [ pts3d[0,i], pts3d[1,i], pts3d[2,i], reflectances[i], color[0], color[1], color[2], pts2d_normed[0,i], pts2d_normed[1,i]  ]
            points.append(point)

    points = np.array(points)
    return points

# update the following directories
IMG_ROOT = '/content/Voxelnet/crop_data/training/image_2/'
PC_ROOT = '/content/Voxelnet/crop_data/training/velodyne/'
CALIB_ROOT = '/content/Voxelnet/crop_data/training/calib/'



for frame in range(0, 7481):
    img_dir = IMG_ROOT + '%06d.png' % frame
    pc_dir = PC_ROOT + '%06d.bin' % frame
    calib_dir = CALIB_ROOT + '%06d.txt' % frame

    points = align_img_and_pc(img_dir, pc_dir, calib_dir)
    
    output_name = str(PC_ROOT) + str(frame) + '.bin'
    points[:,:4].astype('float32').tofile(output_name)

Overwriting /content/Voxelnet/crop_data/crop.py


In [ ]:
!pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 130kB/s 
ERROR: umap-learn 0.4.6 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
#running the crop.py file
!python crop.py

**Creating the validation data**

**1. Create a folder named 'validation'**

**2. copy the content from training to the validation folder**

In [ ]:
!mkdir /content/Voxelnet/crop_data/validation/
%cp -av /content/Voxelnet/crop_data/training /content/Voxelnet/crop_data/validation/

**Splitting the training and validation data**

In [ ]:
import os

lines_train = [line.rstrip('\n') for line in open('/content/Voxelnet/crop_data/training/train.txt')]
lines_val = [line.rstrip('\n') for line in open('/content/Voxelnet/crop_data/training/val.txt')]

for i in lines_train:

  os.remove('/content/Voxelnet/crop_data/training/image_2/'+i+'.png')
  os.remove('/content/Voxelnet/crop_data/training/label_2/'+i+'.txt')
  os.remove('/content/Voxelnet/crop_data/training/velodyne/'+i+'.bin')

for i in lines_val:

  os.remove('/content/Voxelnet/crop_data/validation/image_2/'+i+'.png')
  os.remove('/content/Voxelnet/crop_data/validation/label_2/'+i+'.txt')
  os.remove('/content/Voxelnet/crop_data/validation/velodyne/'+i+'.bin')

**create a .zip folder of the processed and cropped data**

In [ ]:
!zip -r data_lidar.zip /content/Voxelnet/crop_data/

**Move the .zip folder to the drive**


In [ ]:
!mv "/content/VoxelNet/data/data_lidar.zip" "/content/gdrive/My Drive/"